 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 500/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [1]:
import requests
import json
import pandas as pd

 Tutaj zdefiniuj parametry połączenia do API

In [2]:
apikey = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [3]:
list_of_airports_id = pd.read_csv(r'../data/airports.csv')
list_of_airports_id = list(list_of_airports_id['origin_airport_id'])

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [4]:
list_of_airports = []
for airport_id in list_of_airports_id:
    try:
        res = requests.get(f'https://api-datalab.coderslab.com/api/airport/{airport_id}',
                           headers = {'Authorization': f'{apikey}'})
        if res.status_code == 200:
            data = res.json()
            list_of_airports.append(data)
    except:
        pass
airport_df = pd.DataFrame.from_records(list_of_airports)
airport_df.head(10)

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
5,10529,Bradley International,"Hartford, CT","HARTFORD BRADLEY INTERNATIONAL AIRPORT, CT US"
6,10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"
7,10792,Greater Buffalo International,"Buffalo, NY","BUFFALO NIAGARA INTERNATIONAL, NY US"
8,10599,Birmingham Airport,"Birmingham, AL","BIRMINGHAM AIRPORT, AL US"
9,14831,San Jose International,"San Jose, CA","SAN JOSE INTERNATIONAL AIRPORT, CA US"


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [5]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`.

In [7]:
airport_df.to_csv(r'../data/raw/airport_list.csv')

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [55]:
list_of_weather = []
for year in range(2019, 2021):
    for month in range(1, 13):
        if month < 10:
            month = f'0{month}'
        try:
            res = requests.get(f'https://api-datalab.coderslab.com/api/airportWeather',
                               params = {'date': f'{year}-{month}'},
                               headers = {'Authorization': f'{apikey}'})
            if res.status_code == 200:
                data = res.json()
                for el in data:
                    list_of_weather.append(el)
        except:
            pass
        if(year == 2020 and int(month) == 3):
            break
weather_df = pd.DataFrame.from_records(list_of_weather)
weather_df.head(3)

,WT18,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,...,PGTM,WT10,WESD,SN32,SX32,PSUN,TSUN,TOBS,WT07,WT11
0,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [57]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`.

In [59]:
weather_df.to_csv(r'../data/raw/airport_weather.csv')

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft` a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [60]:
list_of_aircrafts = []
try:
    res = requests.get(f'https://api-datalab.coderslab.com/api/aircraft',
                       headers = {'Authorization': f'{apikey}'})
    if res.status_code == 200:
        data = res.json()
        for el in data:
            list_of_aircrafts.append(el)
except:
    pass
aircraft_df = pd.DataFrame.from_records(list_of_aircrafts)
aircraft_df.head(3)

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [61]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`.

In [63]:
aircraft_df.to_csv(r'../data/raw/aircraft.csv')

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.

In [4]:
list_of_flights = []
for airport_id in list_of_airports_id:
    for year in range(2019, 2021):
        for month in range(1, 13):
            if month < 10:
                month = f'0{month}'
            try:
                res = requests.get(f'https://api-datalab.coderslab.com/api/flight',
                                   params = {'date': f'{year}-{month}', 'airportId': f'{airport_id}'},
                                   headers = {'Authorization': f'{apikey}'})
                if res.status_code == 200:
                    data = res.json()
                    for el in data:
                        list_of_flights.append(el)
            except:
                pass
            if(year == 2020 and int(month) == 3):
                break
flight_df = pd.DataFrame.from_records(list_of_flights)
flight_df.head(3)

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,YEAR,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,1,2,9E,N931XJ,3290,10874,10397,600,557.0,...,129.0,100.0,528,3,2019,NaN,NaN,NaN,NaN,NaN
1,1,1,2,OH,N723PS,5495,10874,11057,704,723.0,...,115.0,82.0,394,2,2019,NaN,NaN,NaN,NaN,NaN
2,1,1,2,OH,N525EA,5416,10874,11057,1944,1942.0,...,101.0,96.0,394,2,2019,NaN,NaN,NaN,NaN,NaN


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [5]:
flight_df_expected_shape = (9251880, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`.

In [7]:
flight_df.to_csv(r'../data/raw/flight.csv')

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [8]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.
